In [8]:
import re
import os
import nltk
import string
import mailbox
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from email.header import decode_header
from nltk.tokenize import word_tokenize

dotRegex = r'\.'
digitsRegex = r'[0-9]'
ipAddressRegex = r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'
url_pattern = r'[http[s]]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
dashesRegex = r'-'
specialCharsRegex = r'[()@:%_\+~#?\=;]'
# stop_words = set(stopwords.words('english'))
stemmer = nltk.PorterStemmer()
punctuations = ['!','@','#','$','%','^','&','*','(',')','-','_','=','+',';',':',"'",'"','?','/','<','>','.',',','/','~','`']

def process_text(text):
    text = text.lower()                    #lowercase
    text = re.sub(r'[\n\t\r]', ' ', text)  #remove escape sequences 

    # Replace all numbers in the text with the replacement character
    text = re.sub(url_pattern, " ", text)
    text = re.sub(r"\b\d{1,2}/\d{1,2}/\d{2,4}\b|\b\d{4}-\d{1,2}-\d{1,2}\b", ' ', text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2}(st|nd|rd|th)?\b", ' ', text)

    #remove punctuations
    punctuation = string.punctuation  # Get all punctuation marks
    translator = str.maketrans('', '', punctuation + string.digits)  # Create a translator to remove punctuation and digits
    text = text.translate(translator)  # Remove punctuation and digits using translate()
    return text

In [3]:
t = '> Hi! Is there a command to
insert the signature using a combination of keys and not to have sent the mail to insert it then? I simply put it (them) into my (nmh) component files (components, replcomps, forwcomps and so on). That way you get them when you are editing your message. Also, by using comps files for specific folders you can alter your .sig per folder (and other tricks).  See the docs for (n)mh for all the details. There might (must?) also be a way to get sedit to do it, but I‘ve been using gvim as my exmh message editor for a long time now.  I load it with a command that loads some email-specific settings, eg, to syntax colour-highlight the headers and quoted parts of an email... it would be possible to map some (vim) keys that would add a sig (or even give a selection of sigs to choose from). And there are all sorts of ways to have randomly-chosen sigs... somewhere at rtfm.mit.edu... ok, here we go: rtfm.mit.edu/pub/usenet-by-group/news.answers/signature_finger_faq. (Warning... it\'s old, May 1995). Regards, Ulises'

In [4]:
t

"> Hi! Is there a command to insert the signature using a combination of keys and not to have sent the mail to insert it then? I simply put it (them) into my (nmh) component files (components, replcomps, forwcomps and so on). That way you get them when you are editing your message. Also, by using comps files for specific folders you can alter your .sig per folder (and other tricks).  See the docs for (n)mh for all the details. There might (must?) also be a way to get sedit to do it, but I‘ve been using gvim as my exmh message editor for a long time now.  I load it with a command that loads some email-specific settings, eg, to syntax colour-highlight the headers and quoted parts of an email... it would be possible to map some (vim) keys that would add a sig (or even give a selection of sigs to choose from). And there are all sorts of ways to have randomly-chosen sigs... somewhere at rtfm.mit.edu... ok, here we go: rtfm.mit.edu/pub/usenet-by-group/news.answers/signature_finger_faq. (Warn

In [11]:
t2

' hi is there a command to insert the signature using a combination of keys and not to have sent the mail to insert it then i simply put it them into my nmh component files components replcomps forwcomps and so on that way you get them when you are editing your message also by using comps files for specific folders you can alter your sig per folder and other tricks  see the docs for nmh for all the details there might must also be a way to get sedit to do it but i‘ve been using gvim as my exmh message editor for a long time now  i load it with a command that loads some emailspecific settings eg to syntax colourhighlight the headers and quoted parts of an email it would be possible to map some vim keys that would add a sig or even give a selection of sigs to choose from and there are all sorts of ways to have randomlychosen sigs somewhere at ok here we go warning its old   regards ulises'

In [35]:
def check_link(urls):
    
    if urls is not None:
        if type(urls) != str:
            urls = str(urls)
        urls = urls.split(", ")
        for url in urls:
            # Check if the URL starts with http or https
            if not url.startswith("http://") and not url.startswith("https://"):
                return 1
            # Check if the URL contains IP address instead of domain name
            if re.match("^https?://(?:[0-9]{1,3}\.){3}[0-9]{1,3}(?:/.*)?$", url):
                return 1
            # Check if the URL contains a port number
            if re.search(":[0-9]+", url):
                return 1
            # Check if the URL contains double slashes after the protocol
            if re.search("//+", url.split("://")[1]):
                return 1
            # Check if the URL contains a suspicious top-level domain
            suspicious_tlds = [".zip", ".exe", ".rar", ".scr"]
            if any(url.endswith(tld) for tld in suspicious_tlds):
                return 1

    return 0

In [36]:
def check_ip_in_link(urls):
    
    if urls is not None:
        if type(urls) != str:
            urls = str(urls)
        urls = urls.split(", ")
        for url in urls:
            # Check if the URL contains IP address instead of domain name
            if re.match("^https?://(?:[0-9]{1,3}\.){3}[0-9]{1,3}(?:/.*)?$", url):
                return 1
    return 0

In [37]:
def remove_repeating_characters(string):
    result = ''
    count = 1
    prev_char = None

    for char in string:
        if char == prev_char:
            count += 1
        else:
            count = 1

        if count <= 3:
            result += char

        prev_char = char

    return result
def remove_repeating_strings(string):
    pattern = r'(\b\w+\b)(?=.*\b\1\b)'
    result = re.sub(pattern, '', string)
    return result

In [50]:
import email
from bs4 import BeautifulSoup
import re

urlRegex = r'https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=;]*)'
emailRegex = r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)'
ipAddressRegex = r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'


def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.get_text()
    return text

def read_eml_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as eml_file:
        eml_message = email.message_from_file(eml_file)
        
        
        def check_att(eml_message):
            if eml_message.is_multipart():
                for part in eml_message.walk():
                    if part.get_content_disposition() == 'attachment':
                        return 1
            return 0
        Attachments = check_att(eml_message)
        # ...
        sender = re.findall(emailRegex, eml_message['From'])
        if sender is not None:
            try:
                sender_domain = sender[0].split('@')[-1]
            except:
                sender_domain = None

        
        receiver = re.findall(emailRegex, eml_message['To'])
        if receiver is not None:
            try:
                receiver_domain = receiver[0].split('@')[-1]
            except:
                receiver_domain = None

                
        is_same_domain = 1 if (sender_domain == receiver_domain) else 0

        # Extract the body of the email
        if eml_message.is_multipart():
            for part in eml_message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/html':
                    html_body = part.get_payload(decode=True).decode('utf-8')
                    text_body = extract_text_from_html(html_body)
                    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
                    urls = re.findall(url_pattern, text_body)
                    if urls:
                        urls = ", ".join(urls)
                        has_sus_link = check_link(urls)
                        has_ip_in_link = check_ip_in_link(urls)
                    else:
                        has_sus_link = 0
                        has_ip_in_link = 0
                    
                    text_body = process_text(text_body)
#                     print(text_body)
        else:
            body = eml_message.get_payload(decode=True).decode('utf-8')
            text_body = extract_text_from_html(body)
            url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
            urls = re.findall(url_pattern, text_body)
            if urls:
                urls = ", ".join(urls)
                has_sus_link = check_link(urls)
                has_ip_in_link = check_ip_in_link(urls)
            else:
                has_sus_link = 0
                has_ip_in_link = 0
            text_body = process_text(text_body)
            text_body = remove_repeating_characters(text_body)
#             print(text_body)
            
        
    return text_body, is_same_domain, Attachments, has_sus_link, has_ip_in_link


In [39]:
def get_files_in_directory(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

In [52]:
paths = get_files_in_directory('/home/ec2-user/SageMaker/email_phishing/Huaibo/data/phishing_pot/email')
text_body, is_same_domain, Attachments, has_sus_link, has_ip_in_link = [], [], [], [], []

for path in paths:
    try:
        returns  = read_eml_file(path)
        text_body.append(returns[0])
        is_same_domain.append(returns[1])
        Attachments.append(returns[2])
        has_sus_link.append(returns[3])
        has_ip_in_link.append(returns[4])
    except:
        print("failed")

failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


In [57]:
returns  = read_eml_file("/home/ec2-user/SageMaker/email_phishing/Huaibo/data/emails/email2.eml")
text_body.append(returns[0])
is_same_domain.append(returns[1])
Attachments.append(returns[2])
has_sus_link.append(returns[3])
has_ip_in_link.append(returns[4])

In [59]:
len(text_body)

458

In [60]:
len(has_sus_link)

458

In [61]:
import pandas as pd

data = {
    'text': text_body,
    'is_same_domain': is_same_domain,
    'Attachments': Attachments,
    'has_sus_link': has_sus_link,
    'has_ip_in_link':has_ip_in_link,
    'label':1
}

df = pd.DataFrame(data)

In [63]:
df.text

0         quanto mais pontos melhor   oi rodrigofphot...
1             de best vlagraclalls odell kilohthesher...
2             de miss alice supportenpprogru enviado ...
3      greetings to youplease i want you to confirm i...
4             de best vlagraclalls ivy apqualityartce...
                             ...                        
453       deals            it will take you only a mi...
454     forwarded message from email regyciauydate mo...
455       waarschuwing deze website bevat expliciete ...
456          ‏‎‎‏‎‎p‏‎‎le‏‎‎as‏‎‎e‏‎‎             ‏‎‎...
457     hellouser your one year plan for norton pc pc...
Name: text, Length: 458, dtype: object

In [64]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=0deff56cc4d6dbfcfa38b4187e9025f571a8ffcdaddd307e858360adb3a20339
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6a/67/f8/9cf1a8ff87e0b37f738769df49cc142a655489a6d27b68089f
Successfully built langdetect


In [65]:
from langdetect import detect

def is_english(text):
    try:
        language = detect(text)
        return 1 if language == 'en' else 0
    except:
        return 0

In [83]:
df['is_en'] = df['text'].apply(is_english)

In [89]:
valid = df.loc[df.is_en==1]

In [85]:
valid = valid.sample(n=20)

In [92]:
valid.to_csv('/home/ec2-user/SageMaker/email_phishing/Huaibo/data/phishing_pot.csv', index=False)

In [91]:
valid

,text,is_same_domain,Attachments,has_sus_link,has_ip_in_link,label,is_en,has_sensitive_words
1,de best vlagraclalls odell kilohthesher...,0,0,0,0,1,1,1
2,de miss alice supportenpprogru enviado ...,0,0,0,0,1,1,1
3,greetings to youplease i want you to confirm i...,0,0,0,0,1,1,1
5,hello dear friendplease bear with me i am writ...,0,0,0,0,1,1,1
7,thousands of gorgeous russian girls would l...,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...
444,de consumer rewards supportmaaxcsref...,0,0,0,0,1,1,1
453,deals it will take you only a mi...,0,0,0,0,1,1,1
454,forwarded message from email regyciauydate mo...,0,0,0,0,1,1,1
456,‏‎‎‏‎‎p‏‎‎le‏‎‎as‏‎‎e‏‎‎ ‏‎‎...,0,0,0,0,1,1,1


In [72]:
# has_sensitive_words
with open('/home/ec2-user/SageMaker/email_phishing/Huaibo/data/trigger_words.txt', 'r') as file:
    # Read all the lines into a list
    lines = file.readlines()

# Remove the newline character from each line using strip()
trigger_words = [line.strip().lower() for line in lines]

def check_word(txt):
    for word in trigger_words:
        if word in txt:
            return 1
    return 0

In [73]:
df["has_sensitive_words"] = df["text"].apply(check_word)